In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from keras.applications import vgg16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np
#import matplotlib.pyplot as plt
from keras_vggface import VGGFace
import os
import pandas as pd

Using TensorFlow backend.


In [3]:
# Load the vggface model
vggmodel = VGGFace(include_top=True, weights = 'vggface',input_shape=(224, 224, 3))

In [4]:
vggmodel.summary()

Model: "vggface_vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)     

### Driver Function to Obtain Fc7 Image Features

In [5]:
def fc7(path_to_image):
    
    '''
    Prerequisites:
    Load the vgg_face model
    Input : 
    The path to the image file
    Output :
    The fc7 features of the corresponding image
    '''
    img = image.load_img(path_to_image, target_size=(224, 224,3))
    x = image.img_to_array(img)
    #image_values = cv2.resize(image_values,(224,224))
        
    x = np.asarray(x).reshape((1, 224, 224, 3))
    model_extractfeatures = Model(input=vggmodel.input, output=vggmodel.get_layer('fc7').output)
    fc7_features = model_extractfeatures.predict(x)
    
    return fc7_features[0]

In [7]:
features_vggface = fc7('pic.jpg')

In [8]:
features_vggface.shape

(4096,)

#### Using vggface model features for classification/ regression tasks

In [10]:
from keras.models import Sequential
from keras.layers import Dense

##### Input data is in the form of fc7 layer features from the above vggface neural network
    Number of nodes in the final layer can be changed according to your requirement.
    Here, it's set to 1 signifying a regression task with a linear activation function. 

In [14]:
model = Sequential()
model.add(Dense(100, activation='relu', input_shape=(4096,)))
model.add(Dense(1, activation='linear'))

In [15]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               409700    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 409,801
Trainable params: 409,801
Non-trainable params: 0
_________________________________________________________________


#### Compile the model 

In [30]:
from keras.optimizers import Adam
from keras.metrics import accuracy
from keras.losses import mean_squared_error

In [31]:
model.compile(optimizer='Adam', loss=mean_squared_error, metrics=['accuracy'])

#### Define model callbacks

In [36]:
from keras.callbacks import ReduceLROnPlateau, CSVLogger, ModelCheckpoint

In [37]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)

In [38]:
csv_logger = CSVLogger('training.log')

In [40]:
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint, reduce_lr, csv_logger]

#### Now fit the model upon the training data

In [41]:
#history = model.fit(X_train, y_train, callbacks=callbacks_list)

#### Evaluate the model

In [42]:
#model.evaluate(X_test, y_test)